In [3]:
from torchvision.transforms import transforms
transform=transforms.Compose([transforms.ToTensor(),transforms.Resize(size=(224, 224))])
from torchvision import datasets
BASE_PATH='/home/bubbles/Quantbit Internship/Quality Check/Tomato Color/train'
BASE_PATH_TEST='/home/bubbles/Quantbit Internship/Quality Check/Tomato Color/train'
train=datasets.ImageFolder(root=BASE_PATH,transform=transform)
test=datasets.ImageFolder(root=BASE_PATH_TEST,transform=transform)

In [4]:
from torch.utils.data import DataLoader
train_loader=DataLoader(train, batch_size=16,shuffle=True)
test_loader=DataLoader(test,batch_size=16,shuffle=False)
from torchvision import models
model=models.vgg16('IMAGENET1K_V1')

/home/bubbles/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [5]:
for param in model.features.parameters():
    param.requires_grad = False
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
from torch import nn
# model.fc=nn.Linear(1000,2)
# _modules['6']
model.classifier._modules['6']=nn.Linear(4096,2)
# model.classifier._modules['7']=nn.Sigmoid()

In [7]:
from torch.optim import Adam
clf=model.to('cpu')
opt=Adam(clf.parameters(),lr=1e-5)
loss_fn=nn.CrossEntropyLoss()

In [8]:
import torch
for epoch in range(10):
  total_correct = 0
  total_samples = 0
  for batch in train_loader:
    X,y=batch
    X,y=X.to('cpu'),y.to('cpu')
    yhat=clf(X)
    # print(yhat,y.unsqueeze(1))
    # print(yhat.squeeze(1).float(),y)
    loss=loss_fn(yhat,y)

    opt.zero_grad()
    loss.backward()
    opt.step()
    # print(y,yhat)
    _, predicted = torch.max(yhat, 1)

        # Update the running total of correct predictions and samples
    total_correct += (predicted == y).sum().item()
    total_samples += y.size(0)

    # Calculate the accuracy for this epoch
  accuracy = 100 * total_correct / total_samples
  print(f'Epoch {epoch+1}: Accuracy = {accuracy:.2f}% Loss:{loss}')


/home/bubbles/.local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1: Accuracy = 67.34% Loss:0.7566224336624146
Epoch 2: Accuracy = 80.06% Loss:0.5493375062942505
Epoch 3: Accuracy = 84.68% Loss:0.4046052098274231
Epoch 4: Accuracy = 90.46% Loss:0.16556815803050995
Epoch 5: Accuracy = 92.49% Loss:0.23430418968200684
Epoch 6: Accuracy = 94.22% Loss:0.07999013364315033
Epoch 7: Accuracy = 95.66% Loss:0.15751847624778748
Epoch 8: Accuracy = 96.82% Loss:0.030979808419942856
Epoch 9: Accuracy = 97.69% Loss:0.03028394654393196
Epoch 10: Accuracy = 97.69% Loss:0.15956397354602814


In [9]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images=images.to('cpu')
        labels=labels.to('cpu')
        # calculate outputs by running images through the network
        outputs = clf(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

Accuracy of the network on the test images: 99 %


In [12]:
torch.save("TomatoClassification.pth")

TypeError: save() missing 1 required positional argument: 'f'

In [14]:
file_path = "/home/bubbles/Quantbit Internship/Quality Check/Tomato Color/TomatoClassification.pth"
torch.save(model.state_dict(), file_path)

In [18]:
import torch
from torchvision import transforms
from PIL import Image

# Assuming you have already defined and saved your model
file_path = "/home/bubbles/Quantbit Internship/Quality Check/Tomato Color/TomatoClassification.pth"

# Create an instance of your model
model=models.vgg16('IMAGENET1K_V1') # Replace with the actual class of your model
model.load_state_dict(torch.load(file_path))
model.eval()

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust the size based on your model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess the new image
image_path = "/home/bubbles/Downloads/green.jpeg"
image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

# Make predictions
with torch.no_grad():
    output = model(input_batch)

# Assuming your model outputs class probabilities, you might want to get the predicted class
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()

print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")


RuntimeError: Error(s) in loading state_dict for VGG:
	size mismatch for classifier.6.weight: copying a param with shape torch.Size([2, 4096]) from checkpoint, the shape in current model is torch.Size([1000, 4096]).
	size mismatch for classifier.6.bias: copying a param with shape torch.Size([2]) from checkpoint, the shape in current model is torch.Size([1000]).

In [1]:
import torch
from torchvision import transforms
from PIL import Image
from torchvision import models

# Assuming you have already defined and saved your model
file_path = "/home/bubbles/Quantbit Internship/Quality Check/Tomato Color/TomatoClassification.pth"

# Create an instance of the pre-trained VGG16 model
model = models.vgg16(pretrained=True)  # Replace with the actual class of your model

# Modify the last fully connected layer to match the number of classes in your specific task
num_classes = 2  # Replace with the actual number of classes in your task
model.classifier[6] = torch.nn.Linear(4096, num_classes)

# Load the state dictionary
state_dict = torch.load(file_path)
model.load_state_dict(state_dict)
model.eval()

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Provide the path to the image you want to classify
image_path = "/home/bubbles/Downloads/tomato1.jpeg"
image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

# Make predictions
with torch.no_grad():
    output = model(input_batch)

# Assuming your model outputs class probabilities, you might want to get the predicted class
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()

print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")


/home/bubbles/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bubbles/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predicted class: 0
Class probabilities: tensor([0.7833, 0.2167])
